## Saving and loading models, with application to the EuroSat dataset

Create a neural network that classifies land uses and land covers from satellite imagery. Save your model using Tensorflow's callbacks and reload it later.load in a pre-trained neural network classifier and compare performance with it. 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd

![EuroSAT overview image](data/eurosat_overview.jpg)

#### The EuroSAT dataset

use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake.

Construct a neural network that classifies a satellite image into one of these 10 classes, as well as applying some of the saving and loading techniques.

#### Import the data

The dataset train the model on is a subset of the total data, with 4000 training images and 1000 testing images, with roughly equal numbers of each class. 

In [2]:
def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load(os.path.join(data_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    x_test  = np.load(os.path.join(data_dir, 'x_test.npy'))
    y_test  = np.load(os.path.join(data_dir, 'y_test.npy'))
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#### Build the neural network model

In [4]:
def get_new_model(input_shape):
   
    model=Sequential([
        Conv2D(filters=16,input_shape=input_shape,kernel_size=(3,3),activation='relu',padding='SAME',name='conv_1'),
        Conv2D(filters=8,kernel_size=(3,3),activation='relu',padding='SAME',name='conv_2'),
        MaxPooling2D(pool_size=(8,8),name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32,activation='relu',name='dense_1'),
        Dense(units=10,activation='softmax',name='dense_2')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

#### Compile and evaluate the model

In [5]:
# Create the model

model = get_new_model(x_train[0].shape)

In [6]:
# Evaluate a model's test accuracy
def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [7]:
model.summary()
get_test_accuracy(model, x_test, y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 8, 8, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 18,354
Trainable params: 18,354
Non-trainable params: 0
____________________________________________________

#### Create checkpoints to save model during training

In [8]:
def get_checkpoint_every_epoch():
    return ModelCheckpoint(
        filepath='checkpoints_every_epoch/checkpoint_{epoch:03d}',
        frequency='epoch',
        save_weights_only=True) 


def get_checkpoint_best_only():
       return ModelCheckpoint(
        filepath='checkpoints_best_only/checkpoint',
        frequency='weights',
        save_weights_only=True,
        save_best_only=True,
        monitor='val_accuracy'
    )
    

In [9]:
def get_early_stopping():
    return EarlyStopping(monitor='val_accuracy',patience=3)
    

In [10]:
checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

#### Train model using the callbacks

In [11]:
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 88s 22ms/sample - loss: 1.8731 - accuracy: 0.2865 - val_loss: 1.7019 - val_accuracy: 0.3630
Epoch 2/50
4000/4000 [==============================] - 89s 22ms/sample - loss: 1.5372 - accuracy: 0.4235 - val_loss: 1.4482 - val_accuracy: 0.4100
Epoch 3/50
4000/4000 [==============================] - 86s 22ms/sample - loss: 1.3687 - accuracy: 0.4830 - val_loss: 1.3961 - val_accuracy: 0.4910
Epoch 4/50
4000/4000 [==============================] - 85s 21ms/sample - loss: 1.2365 - accuracy: 0.5435 - val_loss: 1.2739 - val_accuracy: 0.5230
Epoch 5/50
4000/4000 [==============================] - 83s 21ms/sample - loss: 1.1508 - accuracy: 0.5780 - val_loss: 1.2173 - val_accuracy: 0.5450
Epoch 6/50
4000/4000 [==============================] - 82s 20ms/sample - loss: 1.0949 - accuracy: 0.5987 - val_loss: 1.1527 - val_accuracy: 0.5690
Epoch 7/50
4000/4000 [==============================] - 82s 21ms

## accuracy: 88 %


#### Create new instance of model and load on both sets of weights

In [19]:
def get_model_last_epoch(model):
   
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir='checkpoints_every_epoch'))
    return model
    
    
    
def get_model_best_epoch(model):
    
    model.load_weights('checkpoints_best_only/checkpoint')
    return model

In [20]:
model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.747

Model with best epoch weights:
accuracy: 0.758


#### Load, from scratch, a model trained on the EuroSat dataset.

Find another model trained on the `EuroSAT` dataset in `.h5` format. This model is trained on a larger subset of the EuroSAT dataset and has a more complex architecture. The path to the model is `models/EuroSatNet.h5`. See how its testing accuracy compares to my model!

In [23]:
def get_model_eurosatnet():

    model=load_model('models/EuroSatNet.h5')
    return model

In [24]:
model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 16)        6416      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 16)        2320      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 16)        6416      
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 16)      